# Input and output formats

This section provides an overview of the input and output formats
supported by DFTK, usually via integration with a third-party library.

## Reading / writing files supported by AtomsIO
[AtomsIO](https://github.com/mfherbst/AtomsIO.jl) is a Julia package which supports
reading / writing atomistic structures from / to a large range of file formats.
Supported formats include Crystallographic Information Framework (CIF),
XYZ and extxyz files, ASE / Gromacs / LAMMPS / Amber trajectory files
or input files of various other codes (e.g. Quantum Espresso, VASP, ABINIT, CASTEP, …).
The full list of formats is is available in the
[AtomsIO documentation](https://mfherbst.github.io/AtomsIO.jl/stable).

The AtomsIO functionality is split into two packages. The main package, `AtomsIO` itself,
only depends on packages, which are registered in the Julia General package registry.
In contrast `AtomsIOPython` extends `AtomsIO` by parsers depending on python packages,
which are automatically managed via `PythonCall`. While it thus provides the full set of
supported IO formats, this also adds additional practical complications, so some users
may choose not to use `AtomsIOPython`.

As an example we start the calculation of a simple antiferromagnetic iron crystal
using a Quantum-Espresso input file, [Fe_afm.pwi](Fe_afm.pwi).
For more details about calculations on magnetic systems
using collinear spin, see Collinear spin and magnetic systems.

First we parse the Quantum Espresso input file using AtomsIO,
which reads the lattice, atomic positions and initial magnetisation
from the input file and returns it as an
[AtomsBase](https://github.com/JuliaMolSim/AtomsBase.jl) `AbstractSystem`,
the JuliaMolSim community standard for representing atomic systems.

In [1]:
using AtomsIO        # Use Julia-only IO parsers
using AtomsIOPython  # Use python-based IO parsers (e.g. ASE)
system = load_system("Fe_afm.pwi")

FlexibleSystem(Fe₂, periodic = TTT):
    bounding_box      : [ 2.86814        0        0;
                                0  2.86814        0;
                                0        0  2.86814]u"Å"

    Atom(Fe, [       0,        0,        0]u"Å")
    Atom(Fe, [ 1.43407,  1.43407,        0]u"Å")

      .------.  
     /|      |  
    * |      |  
    | |      |  
    | .------.  
    |/  Fe  /   
    Fe-----*    


Next we attach pseudopotential information, since currently the parser is not
yet capable to read this information from the file.

In [2]:
using DFTK
system = attach_psp(system, Fe="hgh/pbe/fe-q16.hgh")

FlexibleSystem(Fe₂, periodic = TTT):
    bounding_box      : [ 2.86814        0        0;
                                0  2.86814        0;
                                0        0  2.86814]u"Å"

    Atom(Fe, [       0,        0,        0]u"Å")
    Atom(Fe, [ 1.43407,  1.43407,        0]u"Å")

      .------.  
     /|      |  
    * |      |  
    | |      |  
    | .------.  
    |/  Fe  /   
    Fe-----*    


Finally we make use of DFTK's AtomsBase integration to run the calculation.

In [3]:
model = model_LDA(system; temperature=0.01)
basis = PlaneWaveBasis(model; Ecut=10, kgrid=(2, 2, 2))
ρ0 = guess_density(basis, system)
scfres = self_consistent_field(basis, ρ=ρ0);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ------
  1   -223.7488925135                    0.22   -6.320    5.6         
  2   -224.1605379552       -0.39       -0.21   -3.325    1.8    196ms
  3   -224.2175997234       -1.24       -1.06   -1.614    3.0    275ms
  4   -224.2198717348       -2.64       -1.46   -1.193    1.1    176ms
  5   -224.2207934150       -3.04       -1.72   -0.824    1.0    169ms
  6   -224.2212080046       -3.38       -1.99   -0.491    1.5    173ms
  7   -224.2213733559       -3.78       -2.35   -0.229    1.7    210ms
  8   -224.2214153261       -4.38       -2.87   -0.078    2.1    195ms
  9   -224.2214205469       -5.28       -3.55   -0.007    2.7    226ms
 10   -224.2214206903       -6.84       -3.81    0.001    2.9    284ms
 11   -224.2214207092       -7.72       -4.02   -0.001    2.7    201ms
 12   -224.2214207177       -8.07       -4.39   -0.000    1.4    209ms
 13   -

## Writing VTK files for visualization
For visualizing the density or the Kohn-Sham orbitals DFTK supports storing
the result of an SCF calculations in the form of VTK files.
These can afterwards be visualized using tools such
as [paraview](https://www.paraview.org/).
Using this feature requires
the [WriteVTK.jl](https://github.com/jipolanco/WriteVTK.jl/) Julia package.

In [4]:
using WriteVTK
save_scfres("iron_afm.vts", scfres; save_ψ=true);

This will save the iron calculation above into the file `iron_afm.vts`,
using `save_ψ=true` to also include the KS orbitals.

## Parsable data-export using json
Many structures in DFTK support the (unexported) `todict` function,
which returns a simplified dictionary representation of the data.

In [5]:
DFTK.todict(scfres.energies)

Dict{String, Float64} with 9 entries:
  "AtomicNonlocal" => -4.34593
  "PspCorrection"  => 7.03001
  "Ewald"          => -161.527
  "total"          => -224.221
  "Entropy"        => -0.0306478
  "Kinetic"        => 75.6631
  "AtomicLocal"    => -161.128
  "Hartree"        => 41.397
  "Xc"             => -21.2803

This in turn can be easily written to disk using a JSON library.
Currently we integrate most closely with `JSON3`,
which is thus recommended.

In [6]:
using JSON3
open("iron_afm_energies.json", "w") do io
    JSON3.pretty(io, DFTK.todict(scfres.energies))
end
println(read("iron_afm_energies.json", String))

{
    "AtomicNonlocal": -4.345930955813521,
    "PspCorrection": 7.03000636467455,
    "Ewald": -161.52650757200072,
    "total": -224.2214207187345,
    "Entropy": -0.030647819521068553,
    "Kinetic": 75.66305034786201,
    "AtomicLocal": -161.12810998313142,
    "Hartree": 41.397007471744395,
    "Xc": -21.28028857254874
}


Once JSON3 is loaded, additionally a convenience function for saving
a summary of `scfres` objects using `save_scfres` is available:

In [7]:
using JSON3
save_scfres("iron_afm.json", scfres)

Similarly a summary of the band data (occupations, eigenvalues, εF, etc.)
for post-processing can be dumped using `save_bands`:

In [8]:
save_bands("iron_afm_scfres.json", scfres)

Notably this function works both for the results obtained
by `self_consistent_field` as well as `compute_bands`:

In [9]:
bands = compute_bands(scfres, kline_density=10)
save_bands("iron_afm_bands.json", bands)

┌ Warning: The provided cell is a supercell: the returned k-path is the standard k-path of the associated primitive cell in the basis of the supercell reciprocal lattice.
│   cell =
│    Spglib.SpglibCell{Float64, Float64, Int64, Float64}
│     lattice:
│       5.41999997388764  0.0  0.0
│       0.0  5.41999997388764  0.0
│       0.0  0.0  5.41999997388764
│     2 atomic positions:
│       0.0  0.0  0.0
│       0.5  0.5  0.0
│     2 atoms:
│       1  1
│     2 magmoms:
│       0.0  0.0
│    
└ @ BrillouinSpglibExt ~/.julia/packages/Brillouin/ESWPN/ext/BrillouinSpglibExt.jl:28


## Writing and reading JLD2 files
The full state of a DFTK self-consistent field calculation can be
stored on disk in form of an [JLD2.jl](https://github.com/JuliaIO/JLD2.jl) file.
This file can be read from other Julia scripts
as well as other external codes supporting the HDF5 file format
(since the JLD2 format is based on HDF5).

In [10]:
using JLD2
save_scfres("iron_afm.jld2", scfres);

Since such JLD2 can also be read by DFTK to start or continue a calculation,
these can also be used for checkpointing or for transferring results
to a different computer.
See Saving SCF results on disk and SCF checkpoints for details.

(Cleanup files generated by this notebook.)

In [11]:
rm.(["iron_afm.vts", "iron_afm.jld2",
     "iron_afm.json", "iron_afm_energies.json", "iron_afm_scfres.json",
     "iron_afm_bands.json"])

6-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing